In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [ ]:
prototxtPath=os.path.sep.join([r'C:\Users\AnkitAnks\Desktop\Mask_Detection','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\AnkitAnks\Desktop\Mask_Detection','res10_300x300_ssd_iter_140000.caffemodel'])


In [ ]:
prototxtPath

In [ ]:
weightsPath

In [ ]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)


In [ ]:
net

In [10]:
model=load_model(r'C:\Users\AnkitAnks\Desktop\Mask_Detection\maskdetector.h5')

In [11]:
model

In [67]:
image=cv2.imread(r'C:\Users\AnkitAnks\Desktop\Mask_Detection\example\06.jpg')
image = cv2.resize(image, (720, 720))

In [68]:
image

array([[[102,  75,   0],
        [105,  80,   0],
        [ 94,  73,   0],
        ...,
        [ 31,  24,   0],
        [ 31,  24,   0],
        [ 31,  24,   0]],

       [[100,  74,   0],
        [ 98,  74,   0],
        [100,  75,   0],
        ...,
        [ 28,  23,   0],
        [ 30,  25,   0],
        [ 29,  24,   0]],

       [[102,  78,   0],
        [104,  77,   0],
        [106,  76,   0],
        ...,
        [ 27,  22,   0],
        [ 29,  24,   0],
        [ 30,  25,   0]],

       ...,

       [[ 36,  35,  21],
        [ 34,  34,  20],
        [ 32,  32,  20],
        ...,
        [ 38,  37,  17],
        [ 38,  38,  17],
        [ 35,  36,  15]],

       [[ 33,  33,  20],
        [ 32,  32,  19],
        [ 33,  33,  21],
        ...,
        [ 41,  40,  20],
        [ 41,  43,  20],
        [ 36,  38,  11]],

       [[ 31,  31,  19],
        [ 31,  31,  19],
        [ 29,  31,  19],
        ...,
        [ 40,  37,  18],
        [ 36,  35,  12],
        [ 38,  40,  10]]

In [69]:
(h,w)=image.shape[:2]

In [70]:
(h,w)

(720, 720)

In [71]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [72]:
blob

array([[[[  -4.,    0.,   -1., ...,  -76.,  -77.,  -74.],
         [  -1.,    4.,    8., ...,  -75.,  -76.,  -73.],
         [   2.,    2.,    6., ...,  -71.,  -73.,  -74.],
         ...,
         [ -69.,  -70.,  -75., ...,  -68.,  -71.,  -73.],
         [ -66.,  -71.,  -77., ...,  -62.,  -65.,  -69.],
         [ -72.,  -74.,  -74., ...,  -61.,  -63.,  -66.]],

        [[-102.,  -96.,  -95., ..., -156., -155., -153.],
         [-103.,  -95.,  -92., ..., -155., -154., -151.],
         [ -99.,  -97.,  -94., ..., -154., -153., -153.],
         ...,
         [-142., -142., -148., ..., -139., -142., -144.],
         [-140., -144., -151., ..., -138., -139., -143.],
         [-145., -145., -148., ..., -135., -137., -137.]],

        [[-123., -123., -123., ..., -123., -123., -123.],
         [-123., -123., -123., ..., -123., -123., -122.],
         [-123., -123., -123., ..., -122., -123., -123.],
         ...,
         [-101., -102., -106., ..., -107., -110., -112.],
         [-100., -104., -1

In [73]:
blob.shape

(1, 3, 300, 300)

In [74]:
net.setInput(blob)
detections=net.forward()

In [75]:
detections

array([[[[0.        , 1.        , 0.97755164, ..., 0.3034145 ,
          0.62870795, 0.44047257],
         [0.        , 1.        , 0.9742991 , ..., 0.23470259,
          0.8092247 , 0.39194   ],
         [0.        , 1.        , 0.96661747, ..., 0.31395584,
          0.42492086, 0.43900007],
         ...,
         [0.        , 1.        , 0.09908321, ..., 0.54478985,
          0.46108517, 0.5938768 ],
         [0.        , 1.        , 0.09899408, ..., 0.7742464 ,
          0.63064015, 0.8914625 ],
         [0.        , 1.        , 0.09881545, ..., 0.50306433,
          0.218674  , 0.55460745]]]], dtype=float32)

In [76]:
#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)

In [77]:
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()
